In [10]:
import os, sys
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import pandapower.topology as top
import networkx as nx
import itertools
import tabulate
from tabulate import tabulate
from itertools import combinations
import matplotlib.pyplot as plt

In [11]:
def generate_network(n):
    if n == 14:
        net = pn.case14()
    elif n == 30:
        # net = pn.case_ieee30()
        net = pn.case30()
    elif n == 39:
        net = pn.case39()
    elif n == 57:
        net = pn.case57()
    elif n == 118:
        net = pn.case118()
    elif n == 145:
        net = pn.case145()
    elif n == 300:
        net = pn.case300()
    else:
        sys.exit('invalid case!')
        
# create the power graph
    G = nx.Graph(top.create_nxgraph(net))
# determine the bus locations
    v = net.bus_geodata.index.values
    x = np.round(net.bus_geodata.x.values, 3)
    y = np.round(net.bus_geodata.y.values, 3)
    pos = dict(zip(v, zip(x, y)))

    # return the network, graph, and its bus locations
    return net, G, pos


In [12]:
nodes=tuple(G.nodes())

In [13]:
#Defining all possible 2 node target motifs
M2_1=nx.path_graph(2)
#Defining all possible 3 node target motifs
M3_1=nx.path_graph(3)
M3_2= nx.complete_graph(3)
#Defining all possible 4 node target motifs
#There are 6 different motifs for subgraphs of 4 nodes.
M4_1=nx.path_graph(4)
M4_2=nx.Graph([(1,2),(2,3),(2,4)])
M4_3=nx.cycle_graph(4)
M4_4=nx.Graph([(1,2),(2,3),(3,4),(2,4)])
M4_5=nx.Graph([(1,2),(2,3),(3,4),(4,1),(2,4)])
M4_6=nx.Graph([(1,2),(2,3),(3,4),(4,1),(2,4),(1,3)])
#Defining all possible 5 node target motifs
#There are 21 different motifs for 5 noded graphs
M5_1=nx.path_graph(5)
M5_2=nx.Graph([(1,2),(1,3),(1,4),(1,5)])
M5_3=nx.Graph([(1,2),(2,3),(2,4),(4,5)])
M5_4=nx.Graph([(1,2),(2,3),(3,4),(4,5),(3,5)])
M5_5=nx.Graph([(1,2),(2,3),(3,4),(4,1),(4,5)])
M5_6=nx.cycle_graph(5)
M5_7=nx.Graph([(1,2),(2,3),(3,4),(4,5),(2,4)])
M5_8=nx.Graph([(1,2),(1,3),(2,4),(2,5),(4,5)])
M5_9=nx.Graph([(1,2),(2,3),(3,4),(4,5),(5,2),(3,5)])
M5_10=nx.Graph([(1,2),(2,3),(3,4),(4,5),(5,1),(1,3)])
M5_11=nx.Graph([(1,2),(2,3),(3,4),(4,5),(2,5),(4,2)])
M5_12=nx.Graph([(1,2),(2,3),(3,4),(4,5),(3,5),(1,3)])
M5_13=nx.Graph([(1,2),(2,3),(3,4),(4,1),(1,5),(5,3)])
M5_14=nx.Graph([(1,2),(1,3),(1,4),(1,5),(2,3),(3,4),(4,5)])
M5_15=nx.Graph([(1,2),(2,3),(3,4),(4,1),(2,4),(1,3),(3,5)])
M5_16=nx.Graph([(1,2),(2,3),(3,4),(4,5),(5,2),(1,4),(2,4)])
M5_17=nx.Graph([(1,2),(2,3),(3,4),(4,1),(1,5),(5,2),(5,4)])
M5_18=nx.Graph([(1,2),(2,3),(3,4),(4,1),(5,1),(5,2),(5,3),(5,4)])
M5_19=nx.Graph([(1,2),(2,3),(3,4),(4,1),(2,4),(1,3),(3,5),(4,5)])
M5_20=nx.Graph([(1,2),(2,3),(3,4),(4,5),(5,2),(1,4),(2,4),(1,3),(5,3)])
M5_21=nx.Graph([(1,2),(2,3),(3,4),(4,5),(5,2),(1,4),(2,4),(1,3),(5,3),(1,5)])

In [14]:
#Creating a list of all motifs that the the input graph will be searched for
M2=[M2_1]
M3=[M3_1,M3_2]
M4=[M4_1,M4_2,M4_3,M4_4,M4_5,M4_6]
M5=[M5_1,M5_2,M5_3,M5_4,M5_5,M5_6,M5_7,M5_8,M5_9,M5_10,M5_11,M5_12,M5_14,M5_15,M5_16,M5_17,M5_18,M5_19,M5_20,M5_21]


In [1]:
prefixes = ['C2_', 'C3_', 'C4_', 'C5_']
suffixes = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

for prefix in prefixes:
    for suffix in suffixes:
        if prefix == 'C2_':
            if suffix != 1:
                break
        elif prefix == 'C3_':
            if suffix > 2:
                break
        elif prefix == 'C4_':
            if suffix > 6:
                break
        exec(f"{prefix}{suffix} = 0")

In [16]:
#Creating a list of all motifs that the the input graph will be searched for
C2=[C2_1]
C3=[C3_1,C3_2]
C4=[C4_1,C4_2,C4_3,C4_4,C4_5,C4_6]
C5=[C5_1,C5_2,C5_3,C5_4,C5_5,C5_6,C5_7,C5_8,C5_9,C5_10,C5_11,C5_12,C5_13,C5_14,C5_15,C5_16,C5_17,C5_18,C5_19,C5_20,C5_21]

In [17]:
# counts the target motif in the main graph G
def count_target(G):
    target = nx.complete_graph(2)
    node_comb = combinations(nodes, len(target.nodes()))
    for sub_nodes in node_comb:
        subg = G.subgraph(sub_nodes)
        if nx.is_connected(subg):
            for motif in M2:
                i = M2.index(motif)
                if nx.is_isomorphic(motif, subg):
                    C2[i] = C2[i] + 1
    target = nx.complete_graph(3)
    node_comb = combinations(nodes, len(target.nodes()))
    for sub_nodes in node_comb:
        subg = G.subgraph(sub_nodes)
        if nx.is_connected(subg):
            for motif in M3:
                i = M3.index(motif)
                if nx.is_isomorphic(motif, subg):
                    C3[i] = C3[i] + 1
    target = nx.complete_graph(4)
    node_comb = combinations(nodes, len(target.nodes()))
    for sub_nodes in node_comb:
        subg = G.subgraph(sub_nodes)
        if nx.is_connected(subg):
            for motif in M4:
                i = M4.index(motif)
                if nx.is_isomorphic(motif, subg):
                    C4[i] = C4[i] + 1
    target = nx.complete_graph(5)
    node_comb = combinations(nodes, len(target.nodes()))
    for sub_nodes in node_comb:
        subg = G.subgraph(sub_nodes)
        if nx.is_connected(subg):
            for motif in M5:
                i = M5.index(motif)
                if nx.is_isomorphic(motif, subg):
                    C5[i] = C5[i] + 1

    df=pd.DataFrame(data=[C2,C3,C4,C5],columns=['M1','M2','M3','M4','M5','M6','M7','M8','M9','M10','M11','M12','M13','M14','M15','M16','M17','M18','M19','M20','M21'], index=['2 Nodes','3 Nodes','4 Nodes','5 Nodes'])
    return df

In [18]:
n = 30
pd.set_option('display.max_columns',25)
net, G, pos = generate_network(n)
df=count_target(G)
df

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18,M19,M20,M21
2 Nodes,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3 Nodes,82,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 Nodes,134,49.0,2.0,37.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5 Nodes,186,14.0,208.0,52.0,7.0,0.0,66.0,52.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# checking motif count of 2 nodes
edges=len(tuple(G.edges()))
print(edges)

41
